In [25]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from skimage.feature import hog
from hog_extractor import HOGExtractor
import random as rand
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from helpers import print_executed
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd

hog_parameters = {
    'color_model': 'hsv',  # hls, hsv, yuv, ycrcb
    'svc_input_size': 64,  #
    'number_of_orientations': 11,  # 6 - 12
    'pixels_per_cell': 16,  # 8, 16
    'cells_per_block': 2,  # 1, 2
    'do_transform_sqrt': True,
}

start_frame = cv2.imread("./dataset/positive/00001/00000_00000.ppm")
extractor = HOGExtractor(hog_parameters, start_frame)
positive_images, negative_images = [], []
    
print_executed()



 EXECUTED BLOCK AT 18:3:3


In [26]:
# read file which describe positive dataset
positive_data_descr = pd.read_csv('dataset/positive/folders_classes.csv', delimiter=';', dtype=object)
positive_data_descr = positive_data_descr.drop('opis', axis=1)

# select only prohibitory signs which are speed limits
positive_data_descr = \
    positive_data_descr.loc[(positive_data_descr['typ'] == 'b') & (positive_data_descr['numer'] == '33')]
print(positive_data_descr, '\n')

# read file which describe types of signs and select its language
sign_description = pd.read_csv('dataset/sign_types.csv', delimiter=';')
sign_description = sign_description.drop('opis', axis=1)
print(sign_description.head(5), '\n')

  folder typ numer
0  00000   b    33
1  00001   b    33
2  00002   b    33
3  00003   b    33
4  00004   b    33
5  00005   b    33
7  00007   b    33
8  00008   b    33 

  typ    description
0   a        warning
1   b    prohibitory
2   c      mandatory
3   d  informational 



In [27]:
print("Loading negative images to memory...")

negative_paths = glob.glob('./dataset/negative/*/*.png')
for path in tqdm(negative_paths):
    image = cv2.imread(path)
    if image.shape[0] >= 30 or image.shape[1] >= 30:
        negative_images.append(image)

# good_width, good_height, _ = negative_images[0].shape # zastanowić się czy to jest potrzebne!!!

print_executed()


  0%|          | 0/9866 [00:00<?, ?it/s]

  1%|          | 95/9866 [00:00<00:10, 949.15it/s]

Loading negative images to memory...


  2%|▏         | 201/9866 [00:00<00:09, 978.76it/s]

  3%|▎         | 327/9866 [00:00<00:09, 1047.64it/s]

...

 97%|█████████▋| 9582/9866 [00:10<00:00, 807.40it/s]

 98%|█████████▊| 9663/9866 [00:10<00:00, 804.18it/s]

 99%|█████████▉| 9744/9866 [00:10<00:00, 791.01it/s]

100%|██████████| 9866/9866 [00:10<00:00, 937.37it/s]


 EXECUTED BLOCK AT 18:3:21


In [47]:
print("Loading positive images to memory...\n")

# for each selected folder
for folder in positive_data_descr['folder']:
    # read all images from it and sort it
    paths = sorted(glob.glob('./dataset/positive/' + folder + '/*.ppm'))
    # read csv which contains bounding boxes of traffic sign
    csv = pd.read_csv('./dataset/positive/' + folder + '/GT-' + folder + '.csv', delimiter=';')
    print("Reading folder: ", folder, '\n')
    for path in tqdm(paths):
        # read name of the image
        name = path.split('/')[4]
        # search for selected image in the csv and flatten that row to list
        row = (csv.loc[csv['Filename'] == name]).values.tolist()
        # read image and cut ROI with traffic sign, then add to list
        image = cv2.imread(path)
        if image.shape[0] >= 35 or image.shape[1] >= 35:
            image = image[row[0][3]:row[0][5], row[0][4]:row[0][6]]
            print(image.shape)
            #positive_images.append(image)
    break

print_executed()


  0%|          | 0/210 [00:00<?, ?it/s]

 35%|███▍      | 73/210 [00:00<00:00, 717.25it/s]

Loading positive images to memory...

Reading folder:  00000 

(23, 24, 3)
(24, 25, 3)
(25, 26, 3)
(26, 27, 3)
(27, 27, 3)
(29, 29, 3)
(31, 31, 3)
(33, 33, 3)
(35, 34, 3)
(36, 36, 3)
(40, 38, 3)
(43, 41, 3)
(45, 44, 3)
(47, 47, 3)
(51, 51, 3)
(57, 57, 3)
(63, 63, 3)
(71, 71, 3)
(81, 80, 3)
(90, 91, 3)
(103, 105, 3)
(120, 123, 3)
(24, 24, 3)
(25, 25, 3)
(25, 25, 3)
(26, 25, 3)
(26, 26, 3)
(27, 27, 3)
(28, 28, 3)
(29, 29, 3)
(30, 30, 3)
(31, 32, 3)
(32, 33, 3)
(34, 35, 3)
(35, 36, 3)
(36, 37, 3)
(38, 39, 3)
(40, 42, 3)
(42, 44, 3)
(44, 46, 3)
(46, 48, 3)
(49, 50, 3)
(25, 25, 3)
(27, 27, 3)
(28, 28, 3)
(29, 29, 3)
(30, 30, 3)
(33, 33, 3)
(35, 35, 3)
(37, 37, 3)
(40, 40, 3)
(42, 42, 3)
(44, 44, 3)
(48, 48, 3)
(52, 52, 3)
(56, 58, 3)
(62, 63, 3)
(69, 70, 3)
(76, 78, 3)
(88, 90, 3)
(101, 103, 3)
(25, 25, 3)
(25, 25, 3)
(26, 26, 3)
(27, 27, 3)
(29, 29, 3)
(30, 30, 3)
(32, 32, 3)
(33, 35, 3)
(36, 36, 3)
(40, 40, 3)
(41, 42, 3)
(43, 44, 3)
(47, 46, 3)
(51, 49, 3)
(54, 51, 3)
(58, 55, 3)
(64, 63

 65%|██████▌   | 137/210 [00:00<00:00, 689.84it/s]

 96%|█████████▌| 201/210 [00:00<00:00, 673.16it/s]

100%|██████████| 210/210 [00:00<00:00, 646.90it/s]

(45, 49, 3)
(46, 50, 3)
(47, 50, 3)
(48, 52, 3)
(49, 53, 3)
(49, 53, 3)
(51, 56, 3)
(52, 57, 3)
(54, 58, 3)
(54, 58, 3)
(55, 58, 3)
(57, 60, 3)
(57, 60, 3)
(57, 61, 3)
(58, 62, 3)
(61, 65, 3)
(61, 65, 3)
(62, 65, 3)
(64, 68, 3)
(65, 70, 3)
(66, 71, 3)
(67, 72, 3)
(68, 74, 3)
(70, 74, 3)
(71, 76, 3)
(74, 79, 3)
(78, 83, 3)
(82, 87, 3)
(88, 91, 3)
(25, 23, 3)
(27, 24, 3)
(27, 25, 3)
(28, 27, 3)
(30, 29, 3)
(32, 30, 3)
(33, 31, 3)
(35, 33, 3)
(37, 35, 3)
(39, 37, 3)
(42, 39, 3)
(45, 44, 3)
(48, 47, 3)
(53, 51, 3)
(57, 54, 3)
(63, 60, 3)
(68, 68, 3)
(78, 77, 3)
(91, 89, 3)
(23, 25, 3)
(24, 26, 3)
(25, 27, 3)
(26, 28, 3)
(27, 29, 3)
(26, 29, 3)
(27, 29, 3)
(29, 31, 3)
(29, 31, 3)
(30, 32, 3)
(32, 34, 3)
(34, 36, 3)
(36, 38, 3)
(37, 39, 3)
(40, 42, 3)
(42, 44, 3)
(43, 45, 3)
(45, 48, 3)
(46, 49, 3)
(50, 53, 3)
(52, 55, 3)
(56, 59, 3)
(60, 63, 3)
(63, 67, 3)
(66, 70, 3)
(69, 73, 3)
(75, 79, 3)
(80, 84, 3)
(86, 90, 3)
(93, 97, 3)

 EXECUTED BLOCK AT 18:17:54


In [46]:
# stack already read images to np array
positive_images1, negative_images1 = np.asarray(positive_images), np.asarray(negative_images)
#total_signs, total_nonsigns = positive_images1.shape[0], negative_images1.shape[0]

print("Positive images shape: ", positive_images1.shape, "type: ", type(positive_images))
print("Negative images shape: ", negative_images1.shape, "type: ", type(negative_images))

positive_features, negative_features = [], []

print("Positive images: ", len(positive_images[0][0].shape), "type: ", type(positive_images))
print("Negative images shape: ", len(negative_images), "type: ", type(negative_images))

Positive images shape:  (11835,) type:  <class 'list'>
Negative images shape:  (9866, 64, 64, 3) type:  <class 'list'>
Positive images:  2 type:  <class 'list'>
Negative images shape:  9866 type:  <class 'list'>


In [58]:
print("Extracting features from traffic signs...")

i = 0
# for img in tqdm(positive_images):
for img in positive_images[:2000]:
    #positive_features.append(extractor.features(img)) 
    #i+=1
    #print(i, len(positive_images))
    
print_executed()


Extracting features...
Traffic signs...
1 5880
2 5880
3 5880
4 5880
5 5880
6 5880
7 5880
8 5880
9 5880
10 5880
11 5880
12 5880
13 5880
14 5880


15 5880
16 5880
17 5880
18 5880
19 5880
20 5880
21 5880
22 5880
23 5880
24 5880
25 5880
26 5880
27 5880
28 5880


29 5880
30 5880
31 5880
32 5880
33 5880
34 5880
35 5880
36 5880
37 5880
38 5880
39 5880
40 5880
41 5880
42 5880
43 5880


44 5880
45 5880
46 5880
47 5880
48 5880
49 5880
50 5880
51 5880
52 5880
53 5880
54 5880
55 5880
56 5880
57 5880
58 5880
59 5880


60 5880
61 5880
62 5880
63 5880
64 5880
65 5880
66 5880
67 5880
68 5880
69 5880
70 5880
71 5880
72 5880
73 5880
74 5880
75 5880
76 5880


...

1928 5880
1929 5880
1930 5880
1931 5880
1932 5880
1933 5880
1934 5880
1935 5880
1936 5880
1937 5880
1938 5880
1939 5880
1940 5880
1941 5880
1942 5880
1943 5880
1944 5880


1945 5880
1946 5880
1947 5880
1948 5880
1949 5880
1950 5880
1951 5880
1952 5880
1953 5880
1954 5880
1955 5880
1956 5880
1957 5880
1958 5880
1959 5880
1960 5880
1961 5880
1962 5880


1963 5880
1964 5880
1965 5880
1966 5880
1967 5880
1968 5880
1969 5880
1970 5880
1971 5880
1972 5880
1973 5880
1974 5880
1975 5880
1976 5880
1977 5880
1978 5880


1979 5880
1980 5880
1981 5880
1982 5880
1983 5880
1984 5880
1985 5880
1986 5880
1987 5880
1988 5880
1989 5880
1990 5880
1991 5880
1992 5880
1993 5880
1994 5880


1995 5880
1996 5880
1997 5880
1998 5880
1999 5880
2000 5880

 EXECUTED BLOCK AT 18:1:53


In [60]:
print("Extracting features from non traffic signs...")

i = 0
# for img in tqdm(negative_images):
for img in negative_images[:2000]:
    negative_features.append(extractor.features(img)) 
    i += 1
    print(i, len(negative_images))
    
print_executed()

Extracting features from non traffic signs...
1 3900
2 3900
3 3900
4 3900
5 3900
6 3900
7 3900
8 3900
9 3900
10 3900
11 3900
12 3900
13 3900


14 3900
15 3900
16 3900
17 3900
18 3900
19 3900
20 3900
21 3900
22 3900
23 3900
24 3900
25 3900
26 3900
27 3900
28 3900


29 3900
30 3900
31 3900
32 3900
33 3900
34 3900
35 3900
36 3900
37 3900
38 3900
39 3900
40 3900
41 3900
42 3900
43 3900
44 3900


45 3900
46 3900
47 3900
48 3900
49 3900
50 3900
51 3900
52 3900
53 3900
54 3900
55 3900
56 3900
57 3900
58 3900
59 3900
60 3900
61 3900
62 3900


63 3900
64 3900
65 3900
66 3900
67 3900
68 3900
69 3900
70 3900
71 3900
72 3900
73 3900
74 3900
75 3900
76 3900
77 3900
78 3900
79 3900


...

1929 3900
1930 3900
1931 3900
1932 3900
1933 3900
1934 3900
1935 3900
1936 3900
1937 3900
1938 3900
1939 3900
1940 3900
1941 3900
1942 3900
1943 3900
1944 3900
1945 3900


1946 3900
1947 3900
1948 3900
1949 3900
1950 3900
1951 3900
1952 3900
1953 3900
1954 3900
1955 3900
1956 3900
1957 3900
1958 3900
1959 3900
1960 3900
1961 3900
1962 3900


1963 3900
1964 3900
1965 3900
1966 3900
1967 3900
1968 3900
1969 3900
1970 3900
1971 3900
1972 3900
1973 3900
1974 3900
1975 3900
1976 3900
1977 3900
1978 3900
1979 3900


1980 3900
1981 3900
1982 3900
1983 3900
1984 3900
1985 3900
1986 3900
1987 3900
1988 3900
1989 3900
1990 3900
1991 3900
1992 3900
1993 3900
1994 3900
1995 3900
1996 3900


1997 3900
1998 3900
1999 3900
2000 3900

 EXECUTED BLOCK AT 18:3:50


In [62]:
positive_features = np.asarray(positive_features)
negative_features = np.asarray(negative_features)

print_executed()


 EXECUTED BLOCK AT 18:5:56


In [67]:
print("Scaling features...")

unscaled_x = np.vstack((positive_features, negative_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
# y = np.hstack((np.ones(total_signs), np.zeros(total_nonsigns)))
y = np.hstack((np.ones(2000), np.zeros(2000)))

print(" x shape: ", x.shape, " y shape: ", y.shape)
print_executed()


Scaling features...
 x shape:  (4000, 1188)  y shape:  (4000,)

 EXECUTED BLOCK AT 18:10:45


In [85]:
print("Training classifier and saving models")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.7, random_state=rand.randint(1, 100))
svc = LinearSVC()
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)
print("Accuracy: ", np.round(accuracy, 4))

now = datetime.datetime.now()
created = str(now.year)+str(now.month)+str(now.day)  # +str(now.hour)+str(now.minute)

joblib.dump(svc, './trained_models/SVC_'+created+'.pkl')
joblib.dump(scaler, './trained_models/scaler_'+created+'.pkl')


print_executed()


Training classifier and saving models
Accuracy:  0.9986

 EXECUTED BLOCK AT 18:29:10


In [86]:
print("Test prediction of single ROI")

f1 = extractor.features(negative_images[rand.randint(0, 2000)])
f2 = extractor.features(positive_images[rand.randint(0, 2000)])
f3 = extractor.features(positive_images[rand.randint(0, 2000)])
f4 = extractor.features(negative_images[rand.randint(0, 2000)])

fA = scaler.transform([f1, f2, f3, f4])
print(svc.predict(fA))


f5 = extractor.features(negative_images[rand.randint(2000, total_nonsigns)])
f6 = extractor.features(positive_images[rand.randint(2000, total_signs)])
f7 = extractor.features(positive_images[rand.randint(2000, total_signs)])
f8 = extractor.features(negative_images[rand.randint(2000, total_nonsigns)])

fB = scaler.transform([f5, f6, f7, f8])
print(svc.predict(fB))



[0. 1. 1. 0.]
[0. 1. 1. 0.]


/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)
/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


In [99]:
print("Test prediction of full image")

test_images = [imageio.imread('./dataset/test_images/test_10.png'), 
              imageio.imread('./dataset/test_images/test_11.png'),
              imageio.imread('./dataset/test_images/test_12.png'),
              imageio.imread('./dataset/test_images/test_13.png'),
              imageio.imread('./dataset/test_images/test_14.png')]

import classifier
from helpers import show_images
cls = classifier.Classifier(svc, scaler)
'''
img1 = imageio.imread('./dataset/test_images/test_10.png'),
img2 = imageio.imread('./dataset/test_images/test_11.png'),
img3 = imageio.imread('./dataset/test_images/test_12.png'),
img4 = imageio.imread('./dataset/test_images/test_13.png'),
img5 = imageio.imread('./dataset/test_images/test_14.png'),
'''

for i in test_images:
    extractor.new_frame(i)
    print("predicion: ", cls.predict(extractor.slice(0, 0, 64, 64)), end="\n")
    rgb_img, a_img, b_img, c_img = extractor.visualize()
    show_images([rgb_img, a_img, b_img, c_img], per_row=4, per_col=1, W=100, H=30)

print_executed()


Test prediction of full image


/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


predicion:  0


/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


predicion:  0


...

/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


predicion:  0


/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


predicion:  1



 EXECUTED BLOCK AT 18:50:58
